In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import sys

In [3]:
num_class = 15

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(32, 16, kernel_size=3, padding=1)        
        self.fc1 = nn.Linear(39 * 16, 30)
        self.out_layer = nn.Linear(30, num_class)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.max_pool1d(x, kernel_size=1)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return F.log_softmax(self.out_layer(x), dim=1)

In [4]:
saved_model=CNN()
saved_model.load_state_dict(torch.load('tuesday_model.pth'))

<All keys matched successfully>

In [5]:
saved_model.eval()

CNN(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=624, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=15, bias=True)
)

In [6]:
# data_path = 'F:/Documents/CRCE/Project/NIDS/dataset/Edge-IIoT/Edge-IIoTset dataset/Attack traffic/DDoS_UDP_Flood_attack.csv'  # Replace with your actual path
# data_path = 'live_dataset_1.csv'  # Replace with your actual path
data_path = 'presentation_dataset.csv'  # Replace with your actual path
df = pd.read_csv(data_path, low_memory=False)
df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.msg,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id
0,"May 3, 2025 09:51:38.509882578 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,33:32:2e:33:30:20:37:35:2e:30:30,3,0,0.0,esp32/Temperature_and_Humidity,30,0,0,0,0
1,"May 3, 2025 09:51:38.509970393 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0,0,0
2,"May 3, 2025 09:51:38.809202939 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,7b:22:77:6c:22:3a:31:35:2c:22:74:73:22:3a:32:7d,3,0,0.0,esp32/tank,10,0,0,0,0
3,"May 3, 2025 09:51:38.809281551 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0,0,0
4,"May 3, 2025 09:51:38.969843434 IST",0,0,0,0,0.0,0,0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0,0,0


In [7]:
# Drop unnecessary columns
drop_columns = [
    "frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4", "arp.dst.proto_ipv4",
    "http.file_data", "http.request.full_uri", "icmp.transmit_timestamp", "http.request.uri.query",
    "tcp.options", "tcp.payload", "tcp.srcport", "tcp.dstport", "udp.port", "mqtt.msg"
]
df.drop(drop_columns, axis=1, inplace=True, errors='ignore')

# Remove NaNs and duplicates
# df.dropna(axis=0, how='any', inplace=True)
# df.drop_duplicates(subset=None, keep="first", inplace=True)

# Strip strings
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

# DO NOT shuffle here (this is for training only)
# df = shuffle(df)

# One-hot encode categorical columns
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name], prefix=name)
    df = pd.concat([df, dummies], axis=1)
    return df.drop(name, axis=1)

for col in ['http.request.method', 'http.referer', 'http.request.version',
            'dns.qry.name.len', 'mqtt.conack.flags', 'mqtt.protoname', 'mqtt.topic']:
    if col in df.columns:
        df = encode_text_dummy(df, col)

# Save 'Attack_type' if present
if 'Attack_type' in df.columns:
    labels = df['Attack_type']
else:
    labels = None

# Keep only numeric columns
df = df.select_dtypes(include=[np.number]).copy()

# Add back label for later comparison
if labels is not None:
    df['Attack_type'] = labels


In [8]:
len(df.columns)
len(df)

3503

In [9]:
# Separate labels and features
if 'Attack_type' in df.columns:
    labels = df['Attack_type']
    df.drop(['Attack_type'], axis=1, inplace=True)
else:
    labels = None  # You might not have labels in inference

# Drop 'Attack_label' if present
df.drop(['Attack_label'], axis=1, inplace=True, errors='ignore')

# Reindex to match training feature order (critical)
feature_columns = joblib.load("feature_columns.pkl")  # saved during training
df = df.reindex(columns=feature_columns, fill_value=0)

# Load the scaler and scale features
scaler = joblib.load("scaler.pkl")
features_scaled = scaler.transform(df)

# Convert to tensor with channel dim
X = torch.tensor(features_scaled, dtype=torch.float32).unsqueeze(1)


NameError: name 'joblib' is not defined

In [10]:
import joblib
import torch

# Load training-time artifacts
feature_columns = joblib.load("feature_columns.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Align current dataframe's columns to match training
df = df.reindex(columns=feature_columns, fill_value=0)

# Apply scaling (again)
features_scaled = scaler.transform(df)

# Convert to tensor
X_attack = torch.tensor(features_scaled, dtype=torch.float32).unsqueeze(1)  # (batch_size, 1, num_features)

# Load the model architecture and weights
saved_model=CNN()
saved_model.load_state_dict(torch.load('tuesday_model.pth'))
saved_model.eval()  # Put model in evaluation mode


CNN(
  (conv1): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=624, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=15, bias=True)
)

In [11]:
with torch.no_grad():
    outputs = saved_model(X_attack)
    _, predicted = torch.max(outputs, 1)
    predicted_labels = label_encoder.inverse_transform(predicted.numpy())

print(predicted_labels)
len(predicted_labels)

['Normal' 'DDoS_UDP' 'Normal' ... 'DDoS_UDP' 'DDoS_UDP' 'DDoS_UDP']


3503

In [12]:
# data_path = 'F:/Documents/CRCE/Project/NIDS/dataset/Edge-IIoT/Edge-IIoTset dataset/Attack traffic/DDoS_UDP_Flood_attack.csv'  # Replace with your actual path
# data_path = 'live_dataset_1.csv'  # Replace with your actual path
data_path = 'presentation_dataset.csv'  # Replace with your actual path
df = pd.read_csv(data_path, low_memory=False)
df.head()

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.msg,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id
0,"May 3, 2025 09:51:38.509882578 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,33:32:2e:33:30:20:37:35:2e:30:30,3,0,0.0,esp32/Temperature_and_Humidity,30,0,0,0,0
1,"May 3, 2025 09:51:38.509970393 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0,0,0
2,"May 3, 2025 09:51:38.809202939 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,7b:22:77:6c:22:3a:31:35:2c:22:74:73:22:3a:32:7d,3,0,0.0,esp32/tank,10,0,0,0,0
3,"May 3, 2025 09:51:38.809281551 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0,0,0
4,"May 3, 2025 09:51:38.969843434 IST",0,0,0,0,0.0,0,0,0.0,0,...,0.0,0,0,0.0,0.0,0,0,0,0,0


In [13]:
df['predicted_type']=predicted_labels

In [14]:
df

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.msgtype,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,predicted_type
0,"May 3, 2025 09:51:38.509882578 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,3,0,0.0,esp32/Temperature_and_Humidity,30,0,0,0,0,Normal
1,"May 3, 2025 09:51:38.509970393 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP
2,"May 3, 2025 09:51:38.809202939 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,3,0,0.0,esp32/tank,10,0,0,0,0,Normal
3,"May 3, 2025 09:51:38.809281551 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP
4,"May 3, 2025 09:51:38.969843434 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,"May 3, 2025 09:55:38.963705294 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP
3499,"May 3, 2025 09:55:38.968857191 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP
3500,"May 3, 2025 09:55:38.973123843 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP
3501,"May 3, 2025 09:55:38.973725179 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0,0.0,0.0,0,0,0,0,0,DDoS_UDP


In [15]:
df['Attack_label'] = (df['predicted_type'] != 'Normal').astype(int)
df

,frame.time,ip.src_host,ip.dst_host,arp.dst.proto_ipv4,arp.opcode,arp.hw.size,arp.src.proto_ipv4,icmp.checksum,icmp.seq_le,icmp.transmit_timestamp,...,mqtt.proto_len,mqtt.protoname,mqtt.topic,mqtt.topic_len,mqtt.ver,mbtcp.len,mbtcp.trans_id,mbtcp.unit_id,predicted_type,Attack_label
0,"May 3, 2025 09:51:38.509882578 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,0,0.0,esp32/Temperature_and_Humidity,30,0,0,0,0,Normal,0
1,"May 3, 2025 09:51:38.509970393 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1
2,"May 3, 2025 09:51:38.809202939 IST",172.20.10.4,172.20.10.3,0,0,0.0,0,0,0.0,0,...,0,0.0,esp32/tank,10,0,0,0,0,Normal,0
3,"May 3, 2025 09:51:38.809281551 IST",172.20.10.3,172.20.10.4,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1
4,"May 3, 2025 09:51:38.969843434 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,"May 3, 2025 09:55:38.963705294 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1
3499,"May 3, 2025 09:55:38.968857191 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1
3500,"May 3, 2025 09:55:38.973123843 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1
3501,"May 3, 2025 09:55:38.973725179 IST",0,0,0,0,0.0,0,0,0.0,0,...,0,0.0,0.0,0,0,0,0,0,DDoS_UDP,1


In [16]:
# df.to_csv('CNN_predictions.csv', index=False)  # Save predictions to a new CSV file
df.to_csv('presentation_CNN_predictions.csv', index=False)  # Save predictions to a new CSV file

In [64]:
X_attack

tensor([[[-3.4373e-03,  2.7638e+02, -1.6633e-01,  ..., -8.1742e-03,
          -7.7131e-03, -7.3712e-03]],

        [[-3.4373e-03, -3.6182e-03, -1.6633e-01,  ..., -8.1742e-03,
          -7.7131e-03, -7.3712e-03]],

        [[-3.4373e-03,  2.7638e+02, -1.6633e-01,  ..., -8.1742e-03,
          -7.7131e-03, -7.3712e-03]],

        ...,

        [[-3.4373e-03, -3.6182e-03, -1.6633e-01,  ..., -8.1742e-03,
          -7.7131e-03, -7.3712e-03]],

        [[-3.4373e-03, -3.6182e-03, -1.6633e-01,  ..., -8.1742e-03,
          -7.7131e-03, -7.3712e-03]],

        [[-3.4373e-03, -3.6182e-03, -1.6633e-01,  ..., -8.1742e-03,
          -7.7131e-03, -7.3712e-03]]])